In [ ]:
import pandas as pd

from pprint import pprint
import pandas as pd

import requests

import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from horangi_authen import HorangiAuthentication

HOSTNAME = HorangiAuthentication().getHostname()
HEADER = HorangiAuthentication().getRequestHeader()

In [ ]:
PROXIES = {
    "https": os.getenv("PROXIES_HTTPS")
}

# Get Compliance

In [ ]:
PARAMS = {
    "severities": "low,medium,high,critical", # "informational"
    "results": "fail", # "fail" or "pass"
    "statuses": "open,needs_review", # Suppressed => "risk_accepted,false_positive,remediated"
    "page_limit": 100
}

In [ ]:
r = requests.get(HOSTNAME + "/v1/cspm/checks", params=PARAMS, headers=HEADER, proxies=PROXIES)
if r.status_code == 200:
    r = r.json()
    results = r["data"]
    
    while not r["links"]["next"] is None:
        r = requests.get(r["links"]["next"], params=PARAMS, headers=HEADER, proxies=PROXIES)

        if r.status_code == 200:
            r = r.json()
            results += r["data"]

In [ ]:
results_df = pd.DataFrame(results)

In [ ]:
# results_df

In [ ]:
all_attributes_df = pd.DataFrame(list(results_df["attributes"]))
all_relationships_df = pd.DataFrame(list(results_df["relationships"]))

results_df.drop(["attributes", "relationships"], axis=1, inplace=True)

In [ ]:
result_tf_df = pd.concat([results_df,all_attributes_df, all_relationships_df], axis=1, join="inner")

In [ ]:
result_tf_df

In [ ]:
# list(result_tf_df["scan"])
# list(result_tf_df["scan_group"])
# list(result_tf_df["rule"])
# list(result_tf_df["controls"])

# Update Status Compliance

In [ ]:
rule_ids = []

In [ ]:
body = {
    "status": "risk_accepted", # open, false_positive, remediated
    "status_remarks": "[Test] Status updated via POST endpoint",
#     "expires_at": "2022-12-10",
#     "severity": "high",
#     "severity_remarks": "Severity updated via POST endpoint",
    "ids": rule_ids
}

In [ ]:
r = requests.post(HOSTNAME + "/v1/cspm/vm/update", params={}, headers=HEADER, proxies=PROXIES)
if r.status_code == 200:
    r = r.json()

In [ ]:
r